# Install Dependencies

In [ ]:
!pip install sktime
!pip install pmdarima
!pip install seaborn

# Import Libraries

In [2]:
import domojupyter as domo
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from sktime.datasets import load_airline
from sktime.utils.plotting import plot_series
from sktime.forecasting.ets import AutoETS
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_percentage_error
from domojupyter.ai import Metric
from domojupyter.ai import ModelTrainingInformation

smape = MeanAbsolutePercentageError(symmetric=True)

# Train Model

In [ ]:
# Read data into the notebook
df = domo.read_dataframe('Home Sales', query='SELECT * FROM table')
df = df[df['Metro Area'] == 'All']

#Prep Data
df.index = df['DATE']
df.index.freq = df.index.inferred_freq

#Prep Outcome
fh = np.arange(1, 25)
y = df['Home Sales']

#Split Data
y_train, y_test = temporal_train_test_split(y, fh=fh)

#Train Model
#seasonality is 12 to inticate monthly data
forecaster = AutoARIMA(sp=12, suppress_warnings=True)

#Trains and Score data for test
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
plot_series(y, y_pred, y_test, labels=["y", "y_pred", "y_test"])

In [12]:
#Create a /models folder at root level
import os
path = "models"
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:

   # Create a new directory because it does not exist
   os.makedirs(path)
   print("The new directory is created!")

#Save model to pickle file
with open('models/arima.mdl', 'wb') as f:
    pickle.dump(forecaster, f)

# Model Metrics, Hyperparameters

In [13]:
#Calculate model metrics
metrics = {
    "validation:mape": smape(y_test, y_pred)
}
now = datetime.now()

#convert metrics to a Domo Class
domo_metrics = {k: Metric(k, v, None, now) for (k,v) in metrics.items()}

hyperparameters = {}

#Configure metrics, hyperparameters
trainingInformation = ModelTrainingInformation(metrics=domo_metrics, hyperparameters=hyperparameters, algorithm="AutoARIMA")

# Define input/output schema

In [ ]:
y_train.head()

In [ ]:
# Convert input to dataframe
y_train_frame = y_train.to_frame()
y_train_frame.reset_index(inplace=True)
y_train_frame = y_train_frame.rename(columns = {'DATE':'Date'})
y_train_frame['Date'] = pd.to_datetime(y_train_frame['Date']).dt.date
y_train_frame.dtypes

#sample dataframe


In [ ]:
y_pred.head()

In [ ]:
# Convert output to dataframe
y_pred.head()
y_pred_frame = y_pred.to_frame()
y_pred_frame.reset_index(inplace=True)
y_pred_frame = y_pred_frame.rename(columns = {'index':'Date OP', 'Home Sales':'Home Sales OP'})
y_pred_frame['Date OP'] = pd.to_datetime(y_pred_frame['Date OP']).dt.date
y_pred_frame.dtypes

# Define input/output schema

In [ ]:
# Define input/ output schema

from domojupyter.ai import CSVModelIOConfiguration, SchemaObject
from domojupyter.ai import ModelTask, ModelTaskType
from domojupyter.ai import ModelIOConfiguration

io = CSVModelIOConfiguration(data_frame=y_train_frame)
op = CSVModelIOConfiguration(data_frame=y_pred_frame)

task = ModelTask(ModelTaskType.OTHER, input_config=io, output_config=op)
task.to_json()

# Review forecast_arima.py file

# Test .py file

In [ ]:
%run forecast_arima.py

import forecast_arima as model
data = None
result = model.EndpointHandler().invoke(data=None)
result

### Add your initials to the model name to differentiate the models
### Ex. My model would be called 'VS | Home Sales - ETS'# Add Model to Domo.AI

In [ ]:
# When a model is created from a workspace for the very first time, it can take about 10 mins. 
# Snapshot Status will change to 'SUCCESS' when done

import domojupyter.ai.model as ml
ml.create_model("VS | Home Sales - ARIMA", "forecast_arima.py", ['models/arima.mdl'], training= trainingInformation, tasks= [task])

# Start Endpoint

# Compare with ETS